<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

----

# Linear Regression Example

------

We will now examine a dataset with Ecommerce Customer Data for a company's website and mobile app. Then we want to see if we can build a regression model that will predict the customer's yearly spend on the company's product.

First thing to do is start a Spark Session

In [1]:
import findspark
findspark.init('/Users/halloran/spark-2.3.1-bin-hadoop2.7/')
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("../../../assets/data/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [4]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [5]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [6]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [7]:
for item in data.head():
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


## Setting Up DataFrame for Machine Learning 

In [8]:
# A few things we need to do before Spark can accept the data!
# It needs to be in the form of two columns
# ("label","features")

# Import VectorAssembler and Vectors
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [11]:
output = assembler.transform(data)

In [12]:
output.select("features").show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [13]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [14]:
final_data = output.select("features",'Yearly Amount Spent')

In [15]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [16]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                354|
|   mean|   497.438570892423|
| stddev|  80.45661144192633|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                146|
|   mean|  503.8614043382034|
| stddev|  76.55510616539509|
|    min|  304.1355915788555|
|    max|  689.7876041747194|
+-------+-------------------+



In [18]:
# Create a Linear Regression Model object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [19]:
# Fit the model to the data and call this model lrModel
lrModel = lr.fit(train_data,)

In [20]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {} Intercept: {}".format(lrModel.coefficients,lrModel.intercept))

Coefficients: [26.01016816468974,38.63744978101556,0.48411986325026335,61.79735266601318] Intercept: -1062.5411598246635


In [21]:
test_results = lrModel.evaluate(test_data)

In [22]:
# Interesting results....
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  0.8015613020422165|
|  11.544546842596787|
| -6.8970976912826245|
| 0.17071554873007244|
|  19.356572653900287|
|   4.641886573900706|
|  1.0465359233510299|
|  -3.539495109251959|
|-0.37056063372540393|
|   2.578757012837798|
| -11.092393091963913|
| -1.9196139363828024|
|  -5.447713847475768|
|  -5.725260838260851|
|   5.416789245260929|
|   17.12523446113613|
|   5.104209056982199|
|   6.244412064470964|
|   8.522145779766504|
|  3.0848402974568216|
+--------------------+
only showing top 20 rows



In [23]:
unlabeled_data = test_data.select('features')

In [24]:
predictions = lrModel.transform(unlabeled_data)

In [25]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.5743636841713...|441.26285245602344|
|[30.7377203726281...| 450.2361953536331|
|[31.1280900496166...| 564.1497844383373|
|[31.2606468698795...| 421.1559157082213|
|[31.3123495994443...|444.23484537404033|
|[31.3584771924370...| 490.5340638755747|
|[31.3895854806643...|409.02307513663186|
|[31.6253601348306...|379.87639586617615|
|[31.6610498227460...|416.72891421362624|
|[31.7366356860502...| 494.3546892426941|
|[31.8093003166791...| 547.8642924548051|
|[31.8530748017465...| 461.2047373987348|
|[31.8745516945853...|397.73295809374326|
|[31.9453957483445...| 662.7451847759128|
|[32.0305497162129...| 588.8576941733509|
|[32.0478146331398...| 480.2643232977073|
|[32.0542618511847...| 556.7704486120008|
|[32.0789475795693...|351.61930657392077|
|[32.1151190660142...| 341.5360543840786|
|[32.1898447292735...|  530.311713489386|
+--------------------+------------

In [26]:
print("RMSE: {}".format(test_results.rootMeanSquaredError))
print("MSE: {}".format(test_results.meanSquaredError))

RMSE: 9.697364939903443
MSE: 94.03888677766851
